In [5]:
# coding: utf-8

In [40]:
import os
import sys
from pathlib import Path
import pandas as pd
import sqlite3
import sqlalchemy as sa
from sqlalchemy import create_engine
from sqlalchemy.sql import select
from datetime import datetime

In [41]:
sys.path.append(r"../src/collectcube")
from db import *

In [42]:
#local_db_path = '../data/example/landcover.db'
out_dir = 'C:/GISprojects/ParaguayTraining'
local_db_path = os.path.join(out_dir, 'py_training.db')

### Check that pixel table already exists

In [5]:
pixdf = check_table(local_db_path,'pixels')

              PID  Center     cent_X     cent_Y   cent_lat  cent_long  \
277510  0031482_4       0  3081000.0 -3094060.0 -24.458177 -56.400685   
277511  0031482_5       0  3081020.0 -3094060.0 -24.458177 -56.400467   
277512  0031482_6       0  3081000.0 -3094070.0 -24.458259 -56.400675   
277513  0031482_7       0  3081010.0 -3094070.0 -24.458259 -56.400566   
277514  0031482_8       0  3081020.0 -3094070.0 -24.458259 -56.400457   

        ransamp  checked   PID0  PID1 sampgroup      rand     rand2 biome  \
277510        1        0  31482     4  GE_extra  0.564274  0.539983    BH   
277511        1        0  31482     5  GE_extra  0.564274  0.539983    BH   
277512        1        0  31482     6  GE_extra  0.564274  0.539983    BH   
277513        1        0  31482     7  GE_extra  0.564274  0.539983    BH   
277514        1        0  31482     8  GE_extra  0.564274  0.539983    BH   

        gridCell  
277510      3867  
277511      3867  
277512      3867  
277513      3867  
277

## Make sql lc table from existing lut.csv

In [32]:
lc_lut = Path("../data/Class_LUT.csv")
make_LC_table_from_lut(lc_lut, local_db_path, treat_existing='fail')

In [6]:
make_simp_LC_table(lc_lut, local_db_path, [0,30,35,40,80,98], treat_existing='fail')

### check if it worked:

In [15]:
lcdf = check_table(local_db_path,'LC')

    LC_UNQ           USE_NAME LC5_name  LC5  LC25
48      68  Trees-Forest_palm  HighVeg   70    68
49      77     Tree-Water_mix  HighVeg   70    77
50      80       Trees-Forest  HighVeg   70    80
51      98           N_NoCrop  unknown   99    98
52      99            unknown  unknown   99    99


### Add other tables to database using sqlalchemy to control foreign keys

In [50]:
def make_db(db_path):
    sql_db_path = 'sqlite:///'+ db_path
    engine = create_engine(sql_db_path, echo=False)
    make_main_support_tables(engine)
    populate_LC5_table(engine)
    
make_db(local_db_path)

added LC5 and empty PixVar table to db


### Check that tables are made and populated

In [6]:
engine = create_engine('sqlite:///'+ local_db_path)
with engine.connect() as conn:
    lc5_table = Table('LC', MetaData(), autoload_with=engine)
    rp = conn.execute(lc5_table.select())
results = rp.fetchall() 
print(results)

[(0, ' ------', ' ------', 0, 0), (1, 'NoVeg', 'NoVeg', 10, 99), (2, 'NoVeg_Bare', 'NoVeg', 10, 2), (3, 'NoVeg_Built', 'NoVeg', 10, 3), (7, 'NoVeg_Water', 'NoVeg', 10, 7), (9, 'Mixed-path', 'NoVeg', 10, 9), (10, 'Cleared', 'LowVeg', 20, 10), (11, 'TreePlant-new', 'LowVeg', 20, 11), (12, 'Grassland-Natural', 'LowVeg', 20, 12), (13, 'Grassland-Managed', 'LowVeg', 20, 13), (15, 'Grassland', 'LowVeg', 20, 99), (17, 'Grassland-Wet', 'LowVeg', 20, 17), (18, 'Mixed-GrassEdge', 'LowVeg', 20, 19), (19, 'Mixed-FieldEdge', 'LowVeg', 20, 19), (20, 'LowVeg', 'LowVeg', 20, 99), (23, 'Crops-horticulture', 'LowVeg', 20, 35), (30, 'L_Crop-Low', 'LowVeg', 20, 99), (31, 'Crops-Soybeans', 'LowVeg', 20, 31), (32, 'Crops-Beans', 'LowVeg', 20, 35), (33, 'Crops-Corn', 'LowVeg', 20, 33), (34, 'Crops-Mandioca', 'LowVeg', 20, 35), (35, 'M_Crops-mix', 'LowVeg', 20, 35), (36, 'Crops-Peanuts', 'LowVeg', 20, 35), (37, 'Crops-Rice', 'LowVeg', 20, 37), (38, 'Crops-Sugar', 'LowVeg', 20, 38), (40, 'D_Crop_Med', 'MedVeg'

In [13]:
with engine.connect() as con:
    df = pd.read_sql_table('LC5', con)
df.head()

,LC5id,LC5type
0,0,------
1,10,NoVeg
2,20,LowVeg
3,40,MedVeg
4,50,HighVeg


In [47]:
with engine.connect() as con:
    df = pd.read_sql_table('PixelVerification', con)
df.head()

,recID,PID,PID0,PID1,imgDate,LC5,LC,HOMONBHD9,ForestProx,WaterProx,...,State,Type,Width,Notes,entry_lev,doubt_CNC,doubt_LC,doubt_LC5,source,Image
0,1,1_0,1,0,2018-01-01,20.0,31,8.0,NaN,NaN,...,None,None,NaN,,1.0,NaN,NaN,NaN,assumed,None
1,2,1_0,1,0,2022-01-01,20.0,31,8.0,NaN,NaN,...,None,None,NaN,,1.0,NaN,NaN,NaN,GE,None
2,3,2_0,2,0,2018-01-01,20.0,31,8.0,NaN,NaN,...,None,None,NaN,,1.0,NaN,NaN,NaN,assumed,None
3,4,2_0,2,0,2022-01-01,20.0,31,8.0,NaN,NaN,...,None,None,NaN,,1.0,NaN,NaN,NaN,GE,None
4,5,3_0,3,0,2018-01-01,20.0,31,8.0,NaN,NaN,...,None,None,NaN,,1.0,NaN,NaN,NaN,assumed,None


In [15]:
## first record:
testDate = datetime(2000,1,1)
with engine.connect() as conn:
    pixvar = Table('PixelVerification', MetaData(), autoload_with=engine)
    query = sa.insert(pixvar).values(PID='0_0', PID0=0, PID1=0, imgDate=testDate,LC5=0,LC=0,HOMONBHD9=8) 
    go = conn.execute(query)
    conn.commit()
    conn.close()
print(go.inserted_primary_key)

(1,)


### Cleaning

##### To delete a table created by mistake WATCH OUT! This will delete everything in one sweep

In [49]:
'''
conn = sqlite3.connect(local_db_path)
c = conn.cursor()
c.execute("DROP TABLE PixelVerification")
conn.commit()
conn.close()
'''

In [51]:
df = pd.read_csv('C:/GISprojects/ParaguayTraining/LatestPixDbInfo.csv')
engine = create_engine('sqlite:///'+ local_db_path)                 
df.to_sql('PixelVerification', con=engine, if_exists='append', index=False)

C:\Users\klobw\AppData\Local\Temp\ipykernel_23556\1107478555.py:1: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('C:/GISprojects/ParaguayTraining/LatestPixDbInfo.csv')


93203